In [3]:
import sys
import os
import mxnet as mx
import mxnet.metric #import EvalMetric
import numpy as np
import logging
# code to automatically download dataset
mxnet_path = os.path.dirname(os.path.abspath(os.path.expanduser(mxnet.__file__)))
sys.path.append(os.path.join(mxnet_path, "../../tests/python/common"))
sys.path.append(os.path.join(mxnet_path, "../../example/python-howto"))
print sys.path
from data import mnist_iterator
import get_data

/Users/olg/Software/mxnet/python/mxnet
['', '/Users/olg/Software/mxnet/python', '/Users/olg/Software/mxnet-notebooks/python', '/Users/olg/anaconda2/lib/python27.zip', '/Users/olg/anaconda2/lib/python2.7', '/Users/olg/anaconda2/lib/python2.7/plat-darwin', '/Users/olg/anaconda2/lib/python2.7/plat-mac', '/Users/olg/anaconda2/lib/python2.7/plat-mac/lib-scriptpackages', '/Users/olg/anaconda2/lib/python2.7/lib-tk', '/Users/olg/anaconda2/lib/python2.7/lib-old', '/Users/olg/anaconda2/lib/python2.7/lib-dynload', '/Users/olg/anaconda2/lib/python2.7/site-packages', '/Users/olg/anaconda2/lib/python2.7/site-packages/Sphinx-1.4.6-py2.7.egg', '/Users/olg/anaconda2/lib/python2.7/site-packages/aeosa', '/Users/olg/anaconda2/lib/python2.7/site-packages/setuptools-27.2.0-py2.7.egg', '/Users/olg/anaconda2/lib/python2.7/site-packages/IPython/extensions', '/Users/olg/.ipython', '/Users/olg/Software/mxnet/python/mxnet/__init__.pyc/../../tests/python/common', '/Users/olg/Software/mxnet/python/mxnet/../../tests

In [6]:
def perDigitMetric(params):
    #print "Metric", params
    for i, cm in enumerate(params.eval_metric.cms):
        print "Label=", i
        print cm

class MNISTPerDigitAccuracy(mx.metric.EvalMetric):
    """Calculate accuracy"""

    def reset(self):
        if hasattr(self, 'cms'):
            for cm in self.cms:
                cm.fill(0)
        

    def __init__(self,sizes):
        super(MNISTPerDigitAccuracy, self).__init__('mnistperdigiaccuracy')
        self.cms = map( lambda x: np.zeros((x,x), dtype=int), sizes )
        self.reset()
        return super(MNISTPerDigitAccuracy, self).reset()


    def update(self, labels, preds):
        mx.metric.check_label_shapes(labels, preds)
        #print labels, preds
        for i in range(len(labels)):
            for label, pred_label in zip(labels[i].asnumpy(), preds[i].asnumpy()):
                pred_label = int(np.argmax(pred_label))
                label = int(label)
#                print "LABEL=", label, "PRED=", pred_label
                self.cms[i][label,pred_label] += 1
                self.sum_metric += (pred_label == label)
                self.num_inst += 1


In [7]:
logging.basicConfig(level=logging.DEBUG)

In [8]:
train, val = mnist_iterator(batch_size=100, input_shape = (784,))

In [9]:
class EvenOdd_iterator(mx.io.DataIter):
    '''multi label ilab iterator'''

    def __init__(self, data_iter):
        super(EvenOdd_iterator, self).__init__()
        self.data_iter = data_iter
        self.batch_size = self.data_iter.batch_size

    @property
    def provide_data(self):
        return self.data_iter.provide_data

    @property
    def provide_label(self):
        #return [mx.io.DataDesc('softmax_label', (self.batch_size,), np.float32)]
        return [mx.io.DataDesc('softmaxdigit_label', (self.batch_size,), np.float32),
                mx.io.DataDesc('softmaxeo_label', (self.batch_size,), np.float32)]

    def hard_reset(self):
        self.data_iter.hard_reset()

    def reset(self):
        self.data_iter.reset()

    def next(self):
        batch = self.data_iter.next()
        #labels= batch.label
        labels = []
        labels.append(batch.label[0]) 
        eolabels = []
        for i in batch.label[0].asnumpy():
            eolabels.append(i%2)
        eolabels = mx.nd.array(np.array(eolabels))
        labels.append(eolabels)
        return mx.io.DataBatch(data=batch.data, label=labels, pad=batch.pad, index=batch.index)


In [10]:
_train = EvenOdd_iterator(train)
_val = EvenOdd_iterator(train)

In [11]:
data = mx.symbol.Variable('data')
fc1 = mx.symbol.FullyConnected(data = data, name='fc1', num_hidden=128)
act1 = mx.symbol.Activation(data = fc1, name='relu1', act_type="relu")
fc2 = mx.symbol.FullyConnected(data = act1, name = 'fc2', num_hidden = 64)
act2 = mx.symbol.Activation(data = fc2, name='relu2', act_type="relu")
fc3digit = mx.symbol.FullyConnected(data = act2, name='fc3', num_hidden=10)
fc3eo = mx.symbol.FullyConnected(data = act2, name='fc3eo', num_hidden=2)

In [12]:
mlp1 = mx.symbol.SoftmaxOutput(data = fc3digit, name = 'softmaxdigit')
mlp2 = mx.symbol.SoftmaxOutput(data = fc3eo, name = 'softmaxeo')
mlp = mx.symbol.Group([mlp1,mlp2])

optimizer_params = (('learning_rate', 0.1),('momentum', 0.9), ('wd', 0.00001))

In [13]:
mod = mx.mod.Module(mlp,label_names=['softmaxdigit_label','softmaxeo_label'])
mod.label_names
mod.fit(_train, 
        eval_data=_val,
        optimizer_params=optimizer_params,
        eval_metric=MNISTPerDigitAccuracy([10,2]),
        eval_end_callback=perDigitMetric,
        num_epoch=3, batch_end_callback=mx.callback.Speedometer(100,100))

INFO:root:Epoch[0] Batch [100]	Speed: 41555.49 samples/sec	Train-mnistperdigiaccuracy=0.389356
INFO:root:Epoch[0] Batch [200]	Speed: 44256.78 samples/sec	Train-mnistperdigiaccuracy=0.595920
INFO:root:Epoch[0] Batch [300]	Speed: 45179.96 samples/sec	Train-mnistperdigiaccuracy=0.705050
INFO:root:Epoch[0] Batch [400]	Speed: 45551.25 samples/sec	Train-mnistperdigiaccuracy=0.763741
INFO:root:Epoch[0] Batch [500]	Speed: 45257.47 samples/sec	Train-mnistperdigiaccuracy=0.801856
INFO:root:Epoch[0] Train-mnistperdigiaccuracy=0.827042
INFO:root:Epoch[0] Time cost=1.352
INFO:root:Epoch[0] Validation-mnistperdigiaccuracy=0.896967


Label= 0
[[5760    0   15    5    8   16   37    5   48   29]
 [   1 6548   24   13   12    3    5   14   90   32]
 [  19   28 5719   44   38    3   37   24   39    7]
 [   3   11  105 5769    2  111    4   24   63   39]
 [   8    2   16    1 5611    0   43    5   10  146]
 [  18   15    7  106   26 5073   36    5  104   31]
 [  30    9   10    2   21   61 5757    0   28    0]
 [   8   19   77   31   49   10    0 5894    8  169]
 [  14   21   49   33   36   18   34    2 5593   51]
 [  11    5    6   43   94   22    1   65   30 5672]]
Label= 1
[[28848   644]
 [  725 29783]]


INFO:root:Epoch[1] Batch [100]	Speed: 44342.60 samples/sec	Train-mnistperdigiaccuracy=0.901802
INFO:root:Epoch[1] Batch [200]	Speed: 45865.85 samples/sec	Train-mnistperdigiaccuracy=0.906445
INFO:root:Epoch[1] Batch [300]	Speed: 43607.40 samples/sec	Train-mnistperdigiaccuracy=0.910580
INFO:root:Epoch[1] Batch [400]	Speed: 43832.53 samples/sec	Train-mnistperdigiaccuracy=0.914147
INFO:root:Epoch[1] Batch [500]	Speed: 45915.41 samples/sec	Train-mnistperdigiaccuracy=0.917522
INFO:root:Epoch[1] Train-mnistperdigiaccuracy=0.920417
INFO:root:Epoch[1] Time cost=1.344
INFO:root:Epoch[1] Validation-mnistperdigiaccuracy=0.934694


Label= 0
[[5864    1    6    0   11    5   23    1    9    3]
 [   0 6667   25    4    5    0    9    5   15   12]
 [  35   10 5846    8   20    0   10   10   17    2]
 [   7   14   87 5708    3  205    4   27   35   41]
 [   6    5   13    0 5728    0   15    4    7   64]
 [  17    3    4   19    9 5302   26    3   22   16]
 [  33    8   22    0   18   21 5810    0    6    0]
 [   6   23   57    5   34    9    1 6061    7   62]
 [  44   25   62   29   16   28   58    2 5571   16]
 [  19    8    2   11  149   41    1   31   15 5672]]
Label= 1
[[29283   209]
 [  717 29791]]


INFO:root:Epoch[2] Batch [100]	Speed: 44036.77 samples/sec	Train-mnistperdigiaccuracy=0.936261
INFO:root:Epoch[2] Batch [200]	Speed: 45418.61 samples/sec	Train-mnistperdigiaccuracy=0.937912
INFO:root:Epoch[2] Batch [300]	Speed: 34347.73 samples/sec	Train-mnistperdigiaccuracy=0.939348
INFO:root:Epoch[2] Batch [400]	Speed: 25767.02 samples/sec	Train-mnistperdigiaccuracy=0.940680
INFO:root:Epoch[2] Batch [500]	Speed: 38850.65 samples/sec	Train-mnistperdigiaccuracy=0.941927
INFO:root:Epoch[2] Train-mnistperdigiaccuracy=0.943147
INFO:root:Epoch[2] Time cost=1.710
INFO:root:Epoch[2] Validation-mnistperdigiaccuracy=0.949512


Label= 0
[[5872    0   12    2    6    1    8    3   16    3]
 [   0 6677   22    5    2    2    4    5   15   10]
 [  16    7 5901    7    3    0    0    9   14    1]
 [   4    4   59 5908    0   82    0   20   38   16]
 [   2    4   61    1 5717    0   17    2   11   27]
 [  11    1    8   39    4 5286   14    2   45   11]
 [  38    3   93    0   18   22 5713    0   31    0]
 [   3   17   67   14   23    3    0 6008    8  122]
 [  19   22   45   24   10    8    6    1 5710    6]
 [   8    5   14   42   87   28    1   18   43 5703]]
Label= 1
[[29385   107]
 [  627 29881]]
